# 1.ライブラリの読み込み
上から順に
* グラフを書くためにmatplotlib
* データの整理のためにpandas
* 数学的な計算のためにnumpy
* 時系列データを扱うためにdatetime
* matplotlibでdatetimeのフォーマットを変えるためにmdates

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.dates as mdates
from scipy import signal

# 2.センサーデータの読みこみと確認
ファイルの読み込みに使用した変数名の命名則については以下の通りである．

例　セッション1のAさんのセンサデータ →　data2_d
　　セッション1のAさんのスケーリングデータ　→　data2_d_label_50

In [2]:
%time
#sensor_dataの読み込み

# data2_d = pd.read_csv('sensor_data/session1/TSND151/raw_data_time/acc_gyr/Conv-mem-A胸-20201111-121953632-ags.csv', encoding="shift-jis")
# data1_b = pd.read_csv('sensor_data/session1/TSND151/raw_data_time/acc_gyr/Conv-mem-B胸-20201111-122231707-ags.csv', encoding="shift-jis")
# data1_c = pd.read_csv('sensor_data/session1/TSND151/raw_data_time/acc_gyr/Conv-mem-C胸-20201111-121953885-ags.csv', encoding="shift-jis")
#data2_d = pd.read_csv('sensor_data/session1/TSND151/raw_data_time/acc_gyr/Conv-mem-D胸-20201111-121954011-ags.csv', encoding="shift-jis")
# data2_d = pd.read_csv('sensor_data/session2/TSND151/raw_data_time/acc_gyr/Conv-mem-A胸-20201111-145759193-ags.csv', encoding="shift-jis")
# data2_b = pd.read_csv('sensor_data/session2/TSND151/raw_data_time/acc_gyr/Conv-mem-B胸-20201111-145733362-ags.csv', encoding="shift-jis")
# data2_d = pd.read_csv('sensor_data/session2/TSND151/raw_data_time/acc_gyr/Conv-mem-C胸-20201111-150018208-ags.csv', encoding="shift-jis")
data2_d = pd.read_csv('sensor_data/session2/TSND151/raw_data_time/acc_gyr/Conv-mem-D胸-20201111-145806707-ags.csv', encoding="shift-jis")

#emotion_labelの読み込み

data2_d_label_50 = pd.read_csv('emotion_label/session2/D/emotion_D_20201111_2_50.csv')
data2_d_label_60 = pd.read_csv('emotion_label/session2/D/emotion_D_20201111_2_60.csv')
data2_d_label_70 = pd.read_csv('emotion_label/session2/D/emotion_D_20201111_2_70.csv')
data2_d_label_80 = pd.read_csv('emotion_label/session2/D/emotion_D_20201111_2_80.csv')

# data2_d_label_50 = pd.read_csv('emotion_label/D/emotion_D_20201111_2_50_es.csv')
# data2_d_label_60 = pd.read_csv('emotion_label/D/emotion_D_20201111_2_60_es.csv')
# data2_d_label_70 = pd.read_csv('emotion_label/D/emotion_D_20201111_2_70_es.csv')
# data2_d_label_80 = pd.read_csv('emotion_label/D/emotion_D_20201111_2_80_es.csv')

Wall time: 0 ns


## 2.1 TSND151加速度，角速度センサデータとスケーリングデータの中身
今回のセンサデータの列はセンサの種類，時間，加速度のx，y，z，角速度x，y，zで構成されており，加速度の単位は0.1mG，角速度の単位は0.01dpsである．

スケーリングデータの列は，レコーディング動画の経過時間，感情のレベル値，発話されたユーザ名となっている．emotion_levelの感情値については今回は以下のように考えた．

-4〜-1→negative  0→neutral  １〜4→positive

In [3]:
data2_d_label_50

,start_time,finish_time,emotion_level,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,3:42,3:46,4,NaN,NaN,NaN
1,3:43,3:47,4,NaN,NaN,NaN
2,3:44,3:48,4,NaN,NaN,NaN
3,3:45,3:49,4,NaN,NaN,NaN
4,3:46,3:50,4,NaN,NaN,NaN
...,...,...,...,...,...,...
630,18:04,18:08,0,NaN,NaN,NaN
631,18:05,18:09,0,NaN,NaN,NaN
632,18:06,18:10,0,NaN,NaN,NaN
633,18:07,18:11,0,NaN,NaN,NaN


In [4]:
#欠損値削除
def dropna(label):
    a = label.dropna(how='all', axis=1)
    return a

data2_d_label_50 = dropna(data2_d_label_50)
data2_d_label_60 = dropna(data2_d_label_60)
data2_d_label_70 = dropna(data2_d_label_70)
data2_d_label_80 = dropna(data2_d_label_80)

# data2_d_label_50 = dropna(data2_d_label_50)
# data2_d_label_60 = dropna(data2_d_label_60)
# data2_d_label_70 = dropna(data2_d_label_70)
# data2_d_label_80 = dropna(data2_d_label_80)

In [5]:
data2_d_label_50

,start_time,finish_time,emotion_level
0,3:42,3:46,4
1,3:43,3:47,4
2,3:44,3:48,4
3,3:45,3:49,4
4,3:46,3:50,4
...,...,...,...
630,18:04,18:08,0
631,18:05,18:09,0
632,18:06,18:10,0
633,18:07,18:11,0


In [6]:
#生データの確認
display(data2_d)
display(data2_d_label_50)

,種別,時刻,加速度X,加速度Y,加速度Z,角速度X,角速度Y,角速度Z
0,ags,14:58:06.760,9890,-65,2472,365,610,-105
1,ags,14:58:06.765,9904,37,2450,226,439,-99
2,ags,14:58:06.770,9936,147,2550,73,402,-87
3,ags,14:58:06.775,9929,242,2665,-140,512,-39
4,ags,14:58:06.780,9936,337,2904,-219,292,-21
...,...,...,...,...,...,...,...,...
497905,ags,15:39:36.525,9367,-63,4931,383,177,95
497906,ags,15:39:36.530,9399,-24,4933,414,110,77
497907,ags,15:39:36.535,9328,59,4970,328,134,-33
497908,ags,15:39:36.540,9201,3,4748,128,463,-172


,start_time,finish_time,emotion_level
0,3:42,3:46,4
1,3:43,3:47,4
2,3:44,3:48,4
3,3:45,3:49,4
4,3:46,3:50,4
...,...,...,...
630,18:04,18:08,0
631,18:05,18:09,0
632,18:06,18:10,0
633,18:07,18:11,0


In [7]:
#合成加速度と合成角速度の算出
def norm_order(sensor_data):
    sensor_data["合成加速度"] = np.sqrt(sensor_data["加速度X"]*sensor_data["加速度X"] + sensor_data["加速度Y"]*sensor_data["加速度Y"] + sensor_data["加速度Z"]*sensor_data["加速度Z"])
    sensor_data["合成角速度"] = np.sqrt(sensor_data["角速度X"]*sensor_data["角速度X"] + sensor_data["角速度Y"]*sensor_data["角速度Y"] + sensor_data["角速度Z"]*sensor_data["角速度Z"])
    
norm_order(data2_d)
# norm_order(data1_b)
# norm_order(data1_c)
# norm_order(data2_d)
# norm_order(data2_d)
# norm_order(data2_b)
# norm_order(data2_d)
# norm_order(data2_d)

In [8]:
data2_d

,種別,時刻,加速度X,加速度Y,加速度Z,角速度X,角速度Y,角速度Z,合成加速度,合成角速度
0,ags,14:58:06.760,9890,-65,2472,365,610,-105,10194.464625,718.574979
1,ags,14:58:06.765,9904,37,2450,226,439,-99,10202.601874,503.585147
2,ags,14:58:06.770,9936,147,2550,73,402,-87,10259.054781,417.734365
3,ags,14:58:06.775,9929,242,2665,-140,512,-39,10283.279146,532.226456
4,ags,14:58:06.780,9936,337,2904,-219,292,-21,10357.165684,365.603610
...,...,...,...,...,...,...,...,...,...,...
497905,ags,15:39:36.525,9367,-63,4931,383,177,95,10585.812156,432.484682
497906,ags,15:39:36.530,9399,-24,4933,414,110,77,10614.907725,435.229824
497907,ags,15:39:36.535,9328,59,4970,328,134,-33,10569.577333,355.849687
497908,ags,15:39:36.540,9201,3,4748,128,463,-172,10353.835714,510.232300


# 3.スケーリングデータに即したセンサデータの抽出
スケーリングデータに即したセンサデータの取得方法は以下の通りである．
1. センサデータ及びスケーリングデータの時刻列をdatetime化
2. timedelta関数を使用して，時刻計算を行い，実験実施日に合わせていく
3. センサデータの時刻列をインデックス化

  時刻列をインデックスにすることで，時間を使ってのデータのスライスが可能となる

4. スケーリングデータの時刻列を文字列型のリストにする
5. ４の操作を行うことでセンサデータからスケーリングデータに即したデータを抽出するのが容易になる

In [9]:
#センサデータの時刻合わせ
def datetime_order(sensor_data):
    sensor_data['時刻'] = pd.to_datetime(sensor_data['時刻'], format='%H:%M:%S.%f')
    sensor_data['時刻'] = sensor_data['時刻'] + dt.timedelta(days=44144)

datetime_order(data2_d)
# datetime_order(data1_b)
# datetime_order(data1_c)
# datetime_order(data2_d)

# datetime_order(data2_d)
# datetime_order(data2_b)
# datetime_order(data2_d)
# datetime_order(data2_d)

In [10]:
data2_d

,種別,時刻,加速度X,加速度Y,加速度Z,角速度X,角速度Y,角速度Z,合成加速度,合成角速度
0,ags,2020-11-11 14:58:06.760,9890,-65,2472,365,610,-105,10194.464625,718.574979
1,ags,2020-11-11 14:58:06.765,9904,37,2450,226,439,-99,10202.601874,503.585147
2,ags,2020-11-11 14:58:06.770,9936,147,2550,73,402,-87,10259.054781,417.734365
3,ags,2020-11-11 14:58:06.775,9929,242,2665,-140,512,-39,10283.279146,532.226456
4,ags,2020-11-11 14:58:06.780,9936,337,2904,-219,292,-21,10357.165684,365.603610
...,...,...,...,...,...,...,...,...,...,...
497905,ags,2020-11-11 15:39:36.525,9367,-63,4931,383,177,95,10585.812156,432.484682
497906,ags,2020-11-11 15:39:36.530,9399,-24,4933,414,110,77,10614.907725,435.229824
497907,ags,2020-11-11 15:39:36.535,9328,59,4970,328,134,-33,10569.577333,355.849687
497908,ags,2020-11-11 15:39:36.540,9201,3,4748,128,463,-172,10353.835714,510.232300


In [11]:
data2_d_label_50['start_time'] = pd.to_datetime(data2_d_label_50['start_time'], format='%M:%S')
data2_d_label_50['finish_time'] = pd.to_datetime(data2_d_label_50['finish_time'], format='%M:%S')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [12]:
data2_d_label_60['start_time'] = pd.to_datetime(data2_d_label_60['start_time'], format='%M:%S')
data2_d_label_60['finish_time'] = pd.to_datetime(data2_d_label_60['finish_time'], format='%M:%S')

In [13]:
data2_d_label_70['start_time'] = pd.to_datetime(data2_d_label_70['start_time'], format='%M:%S')
data2_d_label_70['finish_time'] = pd.to_datetime(data2_d_label_70['finish_time'], format='%M:%S')

In [14]:
data2_d_label_80['start_time'] = pd.to_datetime(data2_d_label_80['start_time'], format='%M:%S')
data2_d_label_80['finish_time'] = pd.to_datetime(data2_d_label_80['finish_time'], format='%M:%S')

In [15]:
# data2_d_label_50['start_time'] = pd.to_datetime(data2_d_label_50['start_time'], format='%M:%S')
# data2_d_label_50['finish_time'] = pd.to_datetime(data2_d_label_50['finish_time'], format='%M:%S')

In [16]:
# data2_d_label_60['start_time'] = pd.to_datetime(data2_d_label_60['start_time'], format='%M:%S')
# data2_d_label_60['finish_time'] = pd.to_datetime(data2_d_label_60['finish_time'], format='%M:%S')

In [17]:
# data2_d_label_70['start_time'] = pd.to_datetime(data2_d_label_70['start_time'], format='%M:%S')
# data2_d_label_70['finish_time'] = pd.to_datetime(data2_d_label_70['finish_time'], format='%M:%S')

In [18]:
# data2_d_label_80['start_time'] = pd.to_datetime(data2_d_label_80['start_time'], format='%M:%S')
# data2_d_label_80['finish_time'] = pd.to_datetime(data2_d_label_80['finish_time'], format='%M:%S')

In [19]:
#編集したデータの確認(確認のためセッション1のAさんのデータを表示している)
display(data2_d)
display(data2_d_label_50)

,種別,時刻,加速度X,加速度Y,加速度Z,角速度X,角速度Y,角速度Z,合成加速度,合成角速度
0,ags,2020-11-11 14:58:06.760,9890,-65,2472,365,610,-105,10194.464625,718.574979
1,ags,2020-11-11 14:58:06.765,9904,37,2450,226,439,-99,10202.601874,503.585147
2,ags,2020-11-11 14:58:06.770,9936,147,2550,73,402,-87,10259.054781,417.734365
3,ags,2020-11-11 14:58:06.775,9929,242,2665,-140,512,-39,10283.279146,532.226456
4,ags,2020-11-11 14:58:06.780,9936,337,2904,-219,292,-21,10357.165684,365.603610
...,...,...,...,...,...,...,...,...,...,...
497905,ags,2020-11-11 15:39:36.525,9367,-63,4931,383,177,95,10585.812156,432.484682
497906,ags,2020-11-11 15:39:36.530,9399,-24,4933,414,110,77,10614.907725,435.229824
497907,ags,2020-11-11 15:39:36.535,9328,59,4970,328,134,-33,10569.577333,355.849687
497908,ags,2020-11-11 15:39:36.540,9201,3,4748,128,463,-172,10353.835714,510.232300


,start_time,finish_time,emotion_level
0,1900-01-01 00:03:42,1900-01-01 00:03:46,4
1,1900-01-01 00:03:43,1900-01-01 00:03:47,4
2,1900-01-01 00:03:44,1900-01-01 00:03:48,4
3,1900-01-01 00:03:45,1900-01-01 00:03:49,4
4,1900-01-01 00:03:46,1900-01-01 00:03:50,4
...,...,...,...
630,1900-01-01 00:18:04,1900-01-01 00:18:08,0
631,1900-01-01 00:18:05,1900-01-01 00:18:09,0
632,1900-01-01 00:18:06,1900-01-01 00:18:10,0
633,1900-01-01 00:18:07,1900-01-01 00:18:11,0


### 補足(実験実施時間)

本実験は，zoomレコーディング動画の経過時間でスケーリングデータのemotion_levelをつけたため動画の開始時刻を使って実際の実験実施時間と合わせる必要がある．

方法としては，経過時間と動画の開始時刻を足して実験実施時間を算出する．

動画の開始時間↓

第1セッション12時34分30秒開始

第2セッション15時11分44秒開始

In [20]:
# data2_d_label_50['start_time'] = data2_d_label_50['start_time'] + dt.timedelta(days=44144, hours=12, minutes=34, seconds=30)
# data2_d_label_50['finish_time'] = data2_d_label_50['finish_time'] + dt.timedelta(days=44144, hours=12, minutes=34, seconds=30)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [21]:
# data2_d_label_60['start_time'] = data2_d_label_60['start_time'] + dt.timedelta(days=44144, hours=12, minutes=34, seconds=30)
# data2_d_label_60['finish_time'] = data2_d_label_60['finish_time'] + dt.timedelta(days=44144, hours=12, minutes=34, seconds=30)

In [22]:
# data2_d_label_70['start_time'] = data2_d_label_70['start_time'] + dt.timedelta(days=44144, hours=12, minutes=34, seconds=30)
# data2_d_label_70['finish_time'] = data2_d_label_70['finish_time'] + dt.timedelta(days=44144, hours=12, minutes=34, seconds=30)

In [23]:
# data2_d_label_80['start_time'] = data2_d_label_80['start_time'] + dt.timedelta(days=44144, hours=12, minutes=34, seconds=30)
# data2_d_label_80['finish_time'] = data2_d_label_80['finish_time'] + dt.timedelta(days=44144, hours=12, minutes=34, seconds=30)

In [24]:
data2_d_label_50['start_time'] = data2_d_label_50['start_time'] + dt.timedelta(days=44144, hours=15, minutes=11, seconds=44)
data2_d_label_50['finish_time'] = data2_d_label_50['finish_time'] + dt.timedelta(days=44144, hours=15, minutes=11, seconds=44)

In [25]:
data2_d_label_60['start_time'] = data2_d_label_60['start_time'] + dt.timedelta(days=44144, hours=15, minutes=11, seconds=44)
data2_d_label_60['finish_time'] = data2_d_label_60['finish_time'] + dt.timedelta(days=44144, hours=15, minutes=11, seconds=44)

In [26]:
data2_d_label_70['start_time'] = data2_d_label_70['start_time'] + dt.timedelta(days=44144, hours=15, minutes=11, seconds=44)
data2_d_label_70['finish_time'] = data2_d_label_70['finish_time'] + dt.timedelta(days=44144, hours=15, minutes=11, seconds=44)

In [27]:
data2_d_label_80['start_time'] = data2_d_label_80['start_time'] + dt.timedelta(days=44144, hours=15, minutes=11, seconds=44)
data2_d_label_80['finish_time'] = data2_d_label_80['finish_time'] + dt.timedelta(days=44144, hours=15, minutes=11, seconds=44)

In [28]:
#編集したデータの確認(確認のためセッション1のAさんのデータを表示している)
display(data2_d_label_80)

,start_time,finish_time,emotion_level
0,2020-11-11 12:38:14,2020-11-11 12:38:18,4
1,2020-11-11 12:38:15,2020-11-11 12:38:19,4
2,2020-11-11 12:38:16,2020-11-11 12:38:20,4
3,2020-11-11 12:38:17,2020-11-11 12:38:21,4
4,2020-11-11 12:38:18,2020-11-11 12:38:22,4
...,...,...,...
565,2020-11-11 12:52:34,2020-11-11 12:52:38,0
566,2020-11-11 12:52:35,2020-11-11 12:52:39,0
567,2020-11-11 12:52:36,2020-11-11 12:52:40,0
568,2020-11-11 12:52:37,2020-11-11 12:52:41,0


In [29]:
#datetime化したラベルデータを保存
# data2_d_label_50.to_csv('emotion_label/label_datetime/emotion_A_20201111_1_datetime.csv', index=False)
# data2_d_label_60.to_csv('emotion_label/label_datetime/emotion_B_20201111_1_datetime.csv', index=False)
# data2_d_label_70.to_csv('emotion_label/label_datetime/emotion_C_20201111_1_datetime.csv', index=False)
# data2_d_label_80.to_csv('emotion_label/label_datetime/emotion_D_20201111_1_datetime.csv', index=False)

# data2_d_label_50.to_csv('emotion_label/label_datetime/emotion_A_20201111_2_datetime.csv', index=False)
# data2_d_label_60.to_csv('emotion_label/label_datetime/emotion_B_20201111_2_datetime.csv', index=False)
# data2_d_label_70.to_csv('emotion_label/label_datetime/emotion_C_20201111_2_datetime.csv', index=False)
# data2_d_label_80.to_csv('emotion_label/label_datetime/emotion_D_20201111_2_datetime.csv', index=False)

In [30]:
#datetime化した加速度及び角速度データを保存
# data2_d.to_csv('sensor_data/session1/TSND151/raw_data_datetime/acc_gyr/Conv-mem-A胸-20201111-121953632-ags_datetime.csv', index=False)
# data1_b.to_csv('sensor_data/session1/TSND151/raw_data_datetime/acc_gyr/Conv-mem-B胸-20201111-122231707-ags_datetime.csv', index=False)
# data1_c.to_csv('sensor_data/session1/TSND151/raw_data_datetime/acc_gyr/Conv-mem-C胸-20201111-121953885-ags_datetime.csv', index=False)
# data2_d.to_csv('sensor_data/session1/TSND151/raw_data_datetime/acc_gyr/Conv-mem-D胸-20201111-121954011-ags_datetime.csv', index=False)

# data2_d.to_csv('sensor_data/session2/TSND151/raw_data_datetime/acc_gyr/Conv-mem-A胸-20201111-145759193-ags_datetime.csv', index=False)
# data2_b.to_csv('sensor_data/session2/TSND151/raw_data_datetime/acc_gyr/Conv-mem-B胸-20201111-145733362-ags_datetime.csv', index=False)
# data2_d.to_csv('sensor_data/session2/TSND151/raw_data_datetime/acc_gyr/Conv-mem-C胸-20201111-150018208-ags_datetime.csv', index=False)
# data2_d.to_csv('sensor_data/session2/TSND151/raw_data_datetime/acc_gyr/Conv-mem-D胸-20201111-145806707-ags_datetime.csv', index=False)

In [31]:
#時刻をインデックスに
data2_d.set_index("時刻", inplace=True)
# data1_b.set_index("時刻", inplace=True)
# data1_c.set_index("時刻", inplace=True)
# data2_d.set_index("時刻", inplace=True)

# data2_d.set_index("時刻", inplace=True)
# data2_b.set_index("時刻", inplace=True)
# data2_d.set_index("時刻", inplace=True)
# data2_d.set_index("時刻", inplace=True)

In [32]:
#編集したデータの確認(確認のためセッション1のAさんのデータを表示している)
display(data2_d)

,種別,加速度X,加速度Y,加速度Z,角速度X,角速度Y,角速度Z,合成加速度,合成角速度
時刻,,,,,,,,,
2020-11-11 14:58:06.760,ags,9890,-65,2472,365,610,-105,10194.464625,718.574979
2020-11-11 14:58:06.765,ags,9904,37,2450,226,439,-99,10202.601874,503.585147
2020-11-11 14:58:06.770,ags,9936,147,2550,73,402,-87,10259.054781,417.734365
2020-11-11 14:58:06.775,ags,9929,242,2665,-140,512,-39,10283.279146,532.226456
2020-11-11 14:58:06.780,ags,9936,337,2904,-219,292,-21,10357.165684,365.603610
...,...,...,...,...,...,...,...,...,...
2020-11-11 15:39:36.525,ags,9367,-63,4931,383,177,95,10585.812156,432.484682
2020-11-11 15:39:36.530,ags,9399,-24,4933,414,110,77,10614.907725,435.229824
2020-11-11 15:39:36.535,ags,9328,59,4970,328,134,-33,10569.577333,355.849687


In [33]:
data2_d_label_50["start_time"] = data2_d_label_50["start_time"].dt.strftime("%Y-%m-%d %H:%M:%S")
data2_d_label_50["finish_time"] = data2_d_label_50["finish_time"].dt.strftime("%Y-%m-%d %H:%M:%S")

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [34]:
data2_d_label_60["start_time"] = data2_d_label_60["start_time"].dt.strftime("%Y-%m-%d %H:%M:%S")
data2_d_label_60["finish_time"] = data2_d_label_60["finish_time"].dt.strftime("%Y-%m-%d %H:%M:%S")

In [35]:
data2_d_label_70["start_time"] = data2_d_label_70["start_time"].dt.strftime("%Y-%m-%d %H:%M:%S")
data2_d_label_70["finish_time"] = data2_d_label_70["finish_time"].dt.strftime("%Y-%m-%d %H:%M:%S")

In [36]:
data2_d_label_80["start_time"] = data2_d_label_80["start_time"].dt.strftime("%Y-%m-%d %H:%M:%S")
data2_d_label_80["finish_time"] = data2_d_label_80["finish_time"].dt.strftime("%Y-%m-%d %H:%M:%S")

In [37]:
# data2_d_label_50["start_time"] = data2_d_label_50["start_time"].dt.strftime("%Y-%m-%d %H:%M:%S")
# data2_d_label_50["finish_time"] = data2_d_label_50["finish_time"].dt.strftime("%Y-%m-%d %H:%M:%S")

In [38]:
# data2_d_label_50["start_time"] = data2_d_label_50["start_time"].dt.strftime("%Y-%m-%d %H:%M:%S")
# data2_d_label_50["finish_time"] = data2_d_label_50["finish_time"].dt.strftime("%Y-%m-%d %H:%M:%S")

In [39]:
# data2_d_label_50["start_time"] = data2_d_label_50["start_time"].dt.strftime("%Y-%m-%d %H:%M:%S")
# data2_d_label_50["finish_time"] = data2_d_label_50["finish_time"].dt.strftime("%Y-%m-%d %H:%M:%S")

In [40]:
# data2_d_label_50["start_time"] = data2_d_label_50["start_time"].dt.strftime("%Y-%m-%d %H:%M:%S")
# data2_d_label_50["finish_time"] = data2_d_label_50["finish_time"].dt.strftime("%Y-%m-%d %H:%M:%S")

In [41]:
#編集したデータの確認(確認のためセッション1のAさんのデータを表示している)
display(data2_d_label_70)

,start_time,finish_time,emotion_level
0,2020-11-11 12:38:13,2020-11-11 12:38:17,4
1,2020-11-11 12:38:14,2020-11-11 12:38:18,4
2,2020-11-11 12:38:15,2020-11-11 12:38:19,4
3,2020-11-11 12:38:16,2020-11-11 12:38:20,4
4,2020-11-11 12:38:17,2020-11-11 12:38:21,4
...,...,...,...
587,2020-11-11 12:52:34,2020-11-11 12:52:38,0
588,2020-11-11 12:52:35,2020-11-11 12:52:39,0
589,2020-11-11 12:52:36,2020-11-11 12:52:40,0
590,2020-11-11 12:52:37,2020-11-11 12:52:41,0


In [42]:
#startとfinishをリスト化
data2_d_50_start_time = list(data2_d_label_50["start_time"])
data2_d_50_finish_time = list(data2_d_label_50["finish_time"])

data2_d_60_start_time = list(data2_d_label_60["start_time"])
data2_d_60_finish_time = list(data2_d_label_60["finish_time"])

data2_d_70_start_time = list(data2_d_label_70["start_time"])
data2_d_70_finish_time = list(data2_d_label_70["finish_time"])

data2_d_80_start_time = list(data2_d_label_80["start_time"])
data2_d_80_finish_time = list(data2_d_label_80["finish_time"])

# data2_d_50_start_time = list(data2_d_label_50["start_time"])
# data2_d_50_finish_time = list(data2_d_label_50["finish_time"])

# data2_d_60_start_time = list(data2_d_label_60["start_time"])
# data2_d_60_finish_time = list(data2_d_label_60["finish_time"])

# data2_d_70_start_time = list(data2_d_label_70["start_time"])
# data2_d_70_finish_time = list(data2_d_label_70["finish_time"])

# data2_d_80_start_time = list(data2_d_label_80["start_time"])
# data2_d_80_finish_time = list(data2_d_label_80["finish_time"])

In [43]:
def sensor_segmentation_order(data, data_start_time, data_finish_time, data_label):
    a = [data[data_start_time[i]:data_finish_time[i]]for i in range(0, len(data_label), 1)]
    return a

data2_d_record_1 = sensor_segmentation_order(data2_d, data2_d_50_start_time, data2_d_50_finish_time, data2_d_label_50)
data2_d_record_2 = sensor_segmentation_order(data2_d, data2_d_60_start_time, data2_d_60_finish_time, data2_d_label_60)
data2_d_record_3 = sensor_segmentation_order(data2_d, data2_d_70_start_time, data2_d_70_finish_time, data2_d_label_70)
data2_d_record_4 = sensor_segmentation_order(data2_d, data2_d_80_start_time, data2_d_80_finish_time, data2_d_label_80)

# data2_d_record_1 = sensor_segmentation_order(data2_d, data2_d_50_start_time, data2_d_50_finish_time, data2_d_label_50)
# data2_d_record_2 = sensor_segmentation_order(data2_d, data2_d_60_start_time, data2_d_60_finish_time, data2_d_label_60)
# data2_d_record_3 = sensor_segmentation_order(data2_d, data2_d_70_start_time, data2_d_70_finish_time, data2_d_label_70)
# data2_d_record_4 = sensor_segmentation_order(data2_d, data2_d_80_start_time, data2_d_80_finish_time, data2_d_label_80)

# 4.特徴量算出
感情の識別を行うため学習器作成のために特徴量の算出を行う．今回特徴量としたのは，加速度の各軸及び角速度の各軸における時間領域での平均値，分散値，最大値，最小値及び，周波数領域での特徴量である最大ピーク周波数，最大ピーク値である．プログラムの処理手順を以下に示す．

1. 平均値の算出
2. 分散値の算出
3. 最大値の算出
4. 最小値の算出
5. 最大ピーク周波数及び最大ピーク値の算出
5. 全特徴量の数値を表記したデータフレームを作成

## 4.1 平均値算出
### 4.1.1 加速度

In [44]:
#加速度x,y,z,norm平均値
def acc_gyr_mean_order(data_record, data_label):
    a = [np.mean(data_record[i]["加速度X"]) for i in range(0, len(data_label), 1)]
    b = [np.mean(data_record[i]["加速度Y"]) for i in range(0, len(data_label), 1)]
    c = [np.mean(data_record[i]["加速度Z"]) for i in range(0, len(data_label), 1)]
    d = [np.mean(data_record[i]["合成加速度"]) for i in range(0, len(data_label), 1)]
    a = pd.DataFrame(a)
    b = pd.DataFrame(b)
    c = pd.DataFrame(c)
    d = pd.DataFrame(d)
    acc_mean_list = pd.concat([a, b, c], axis=1, ignore_index=True)
    acc_mean_list.columns = ["accx_mean", "accy_mean", "accz_mean"]
#     acc_mean_list = pd.concat([a, b, c, d], axis=1, ignore_index=True)
#     acc_mean_list.columns = ["accx_mean", "accy_mean", "accz_mean", "accn_mean"]
    e = [np.mean(data_record[i]["角速度X"]) for i in range(0, len(data_label), 1)]
    f = [np.mean(data_record[i]["角速度Y"]) for i in range(0, len(data_label), 1)]
    g = [np.mean(data_record[i]["角速度Z"]) for i in range(0, len(data_label), 1)]
    h = [np.mean(data_record[i]["合成角速度"]) for i in range(0, len(data_label), 1)]
    e = pd.DataFrame(e)
    f = pd.DataFrame(f)
    g = pd.DataFrame(g)
    h = pd.DataFrame(h)
    gyr_mean_list = pd.concat([e, f, g], axis=1, ignore_index=True)
    gyr_mean_list.columns = ["gyrx_mean", "gyry_mean", "gyrz_mean"]
#     gyr_mean_list = pd.concat([e, f, g, h], axis=1, ignore_index=True)
#     gyr_mean_list.columns = ["gyrx_mean", "gyry_mean", "gyrz_mean", "gyrn_mean"]
    mean_list = pd.concat([acc_mean_list, gyr_mean_list], axis=1, ignore_index=False)
    return mean_list

data2_d_50_record_mean = acc_gyr_mean_order(data2_d_record_1, data2_d_label_50)
data2_d_60_record_mean = acc_gyr_mean_order(data2_d_record_2, data2_d_label_60)
data2_d_70_record_mean = acc_gyr_mean_order(data2_d_record_3, data2_d_label_70)
data2_d_80_record_mean = acc_gyr_mean_order(data2_d_record_4, data2_d_label_80)

# data2_d_50_record_mean = acc_gyr_mean_order(data2_d_record_1, data2_d_label_50)
# data2_d_60_record_mean = acc_gyr_mean_order(data2_d_record_2, data2_d_label_60)
# data2_d_70_record_mean = acc_gyr_mean_order(data2_d_record_3, data2_d_label_70)
# data2_d_80_record_mean = acc_gyr_mean_order(data2_d_record_4, data2_d_label_80)

# a = data2_d_50_record_mean
# b = data2_d_60_record_mean
# c = data2_d_70_record_mean
# d = data2_d_80_record_mean

# e = data2_d_50_record_mean
# f = data2_d_60_record_mean
# g = data2_d_70_record_mean
# h = data2_d_80_record_mean

# mean_list = pd.concat([a, b, c, d], ignore_index=True)
# mean_list

In [45]:
#加速度x,y,z,norm分散値
def acc_gyr_var_order(data_record, data_label):
    a = [np.var(data_record[i]["加速度X"]) for i in range(0, len(data_label), 1)]
    b = [np.var(data_record[i]["加速度Y"]) for i in range(0, len(data_label), 1)]
    c = [np.var(data_record[i]["加速度Z"]) for i in range(0, len(data_label), 1)]
    d = [np.var(data_record[i]["合成加速度"]) for i in range(0, len(data_label), 1)]
    a = pd.DataFrame(a)
    b = pd.DataFrame(b)
    c = pd.DataFrame(c)
    d = pd.DataFrame(d)
    acc_var_list = pd.concat([a, b, c], axis=1, ignore_index=True)
    acc_var_list.columns = ["accx_var", "accy_var", "accz_var"]
#     acc_var_list = pd.concat([a, b, c, d], axis=1, ignore_index=True)
#     acc_var_list.columns = ["accx_var", "accy_var", "accz_var", "accn_var"]
    e = [np.var(data_record[i]["角速度X"]) for i in range(0, len(data_label), 1)]
    f = [np.var(data_record[i]["角速度Y"]) for i in range(0, len(data_label), 1)]
    g = [np.var(data_record[i]["角速度Z"]) for i in range(0, len(data_label), 1)]
    h = [np.var(data_record[i]["合成角速度"]) for i in range(0, len(data_label), 1)]
    e = pd.DataFrame(e)
    f = pd.DataFrame(f)
    g = pd.DataFrame(g)
    h = pd.DataFrame(h)
    gyr_var_list = pd.concat([e, f, g], axis=1, ignore_index=True)
    gyr_var_list.columns = ["gyrx_var", "gyry_var", "gyrz_var"]
#     gyr_var_list = pd.concat([e, f, g, h], axis=1, ignore_index=True)
#     gyr_var_list.columns = ["gyrx_var", "gyry_var", "gyrz_var", "gyrn_var"]
    var_list = pd.concat([acc_var_list, gyr_var_list], axis=1, ignore_index=False)
    return var_list

data2_d_50_record_var = acc_gyr_var_order(data2_d_record_1, data2_d_label_50)
data2_d_60_record_var = acc_gyr_var_order(data2_d_record_2, data2_d_label_60)
data2_d_70_record_var = acc_gyr_var_order(data2_d_record_3, data2_d_label_70)
data2_d_80_record_var = acc_gyr_var_order(data2_d_record_4, data2_d_label_80)

# data2_d_50_record_var = acc_gyr_var_order(data2_d_record_1, data2_d_label_50)
# data2_d_60_record_var = acc_gyr_var_order(data2_d_record_2, data2_d_label_60)
# data2_d_70_record_var = acc_gyr_var_order(data2_d_record_3, data2_d_label_70)
# data2_d_80_record_var = acc_gyr_var_order(data2_d_record_4, data2_d_label_80)

# a = data2_d_50_record_var
# b = data2_d_60_record_var
# c = data2_d_70_record_var
# d = data2_d_80_record_var

# e = data2_d_50_record_var
# f = data2_d_60_record_var
# g = data2_d_70_record_var
# h = data2_d_80_record_var

# var_list = pd.concat([a, b, c, d, e, f, g, h], ignore_index=True)
# var_list

In [46]:
#加速度x,y,z,norm分散値
def acc_gyr_max_order(data_record, data_label):
    a = [np.max(data_record[i]["加速度X"]) for i in range(0, len(data_label), 1)]
    b = [np.max(data_record[i]["加速度Y"]) for i in range(0, len(data_label), 1)]
    c = [np.max(data_record[i]["加速度Z"]) for i in range(0, len(data_label), 1)]
    d = [np.max(data_record[i]["合成加速度"]) for i in range(0, len(data_label), 1)]
    a = pd.DataFrame(a)
    b = pd.DataFrame(b)
    c = pd.DataFrame(c)
    d = pd.DataFrame(d)
    acc_max_list = pd.concat([a, b, c], axis=1, ignore_index=True)
    acc_max_list.columns = ["accx_max", "accy_max", "accz_max"]
#     acc_max_list = pd.concat([a, b, c, d], axis=1, ignore_index=True)
#     acc_max_list.columns = ["accx_max", "accy_max", "accz_max", "accn_max"]
    e = [np.max(data_record[i]["角速度X"]) for i in range(0, len(data_label), 1)]
    f = [np.max(data_record[i]["角速度Y"]) for i in range(0, len(data_label), 1)]
    g = [np.max(data_record[i]["角速度Z"]) for i in range(0, len(data_label), 1)]
    h = [np.max(data_record[i]["合成角速度"]) for i in range(0, len(data_label), 1)]
    e = pd.DataFrame(e)
    f = pd.DataFrame(f)
    g = pd.DataFrame(g)
    h = pd.DataFrame(h)
    gyr_max_list = pd.concat([e, f, g], axis=1, ignore_index=True)
    gyr_max_list.columns = ["gyrx_max", "gyry_max", "gyrz_max"]
#     gyr_max_list = pd.concat([e, f, g, h], axis=1, ignore_index=True)
#     gyr_max_list.columns = ["gyrx_max", "gyry_max", "gyrz_max", "gyrn_max"]
    max_list = pd.concat([acc_max_list, gyr_max_list], axis=1, ignore_index=False)
    return max_list

data2_d_50_record_max = acc_gyr_max_order(data2_d_record_1, data2_d_label_50)
data2_d_60_record_max = acc_gyr_max_order(data2_d_record_2, data2_d_label_60)
data2_d_70_record_max = acc_gyr_max_order(data2_d_record_3, data2_d_label_70)
data2_d_80_record_max = acc_gyr_max_order(data2_d_record_4, data2_d_label_80)

# data2_d_50_record_max = acc_gyr_max_order(data2_d_record_1, data2_d_label_50)
# data2_d_50_record_max = acc_gyr_max_order(data2_d_record_2, data2_d_label_60)
# data2_d_50_record_max = acc_gyr_max_order(data2_d_record_3, data2_d_label_70)
# data2_d_50_record_max = acc_gyr_max_order(data2_d_record_4, data2_d_label_80)

# a = data2_d_50_record_max
# b = data2_d_60_record_max
# c = data2_d_70_record_max
# d = data2_d_80_record_max

# e = data2_d_50_record_max
# f = data2_d_60_record_max
# g = data2_d_70_record_max
# h = data2_d_80_record_max

# max_list = pd.concat([a, b, c, d, e, f, g, h], ignore_index=True)
# max_list

In [47]:
#加速度x,y,z,norm分散値
def acc_gyr_min_order(data_record, data_label):
    a = [np.min(data_record[i]["加速度X"]) for i in range(0, len(data_label), 1)]
    b = [np.min(data_record[i]["加速度Y"]) for i in range(0, len(data_label), 1)]
    c = [np.min(data_record[i]["加速度Z"]) for i in range(0, len(data_label), 1)]
    d = [np.min(data_record[i]["合成加速度"]) for i in range(0, len(data_label), 1)]
    a = pd.DataFrame(a)
    b = pd.DataFrame(b)
    c = pd.DataFrame(c)
    d = pd.DataFrame(d)
    acc_min_list = pd.concat([a, b, c], axis=1, ignore_index=True)
    acc_min_list.columns = ["accx_min", "accy_min", "accz_min"]
#     acc_min_list = pd.concat([a, b, c, d], axis=1, ignore_index=True)
#     acc_min_list.columns = ["accx_min", "accy_min", "accz_min", "accn_min"]
    e = [np.min(data_record[i]["角速度X"]) for i in range(0, len(data_label), 1)]
    f = [np.min(data_record[i]["角速度Y"]) for i in range(0, len(data_label), 1)]
    g = [np.min(data_record[i]["角速度Z"]) for i in range(0, len(data_label), 1)]
    h = [np.min(data_record[i]["合成角速度"]) for i in range(0, len(data_label), 1)]
    e = pd.DataFrame(e)
    f = pd.DataFrame(f)
    g = pd.DataFrame(g)
    h = pd.DataFrame(h)
    gyr_min_list = pd.concat([e, f, g], axis=1, ignore_index=True)
    gyr_min_list.columns = ["gyrx_min", "gyry_min", "gyrz_min"]
#     gyr_min_list = pd.concat([e, f, g, h], axis=1, ignore_index=True)
#     gyr_min_list.columns = ["gyrx_min", "gyry_min", "gyrz_min", "gyrn_min"]
    min_list = pd.concat([acc_min_list, gyr_min_list], axis=1, ignore_index=False)
    return min_list

data2_d_50_record_min = acc_gyr_min_order(data2_d_record_1, data2_d_label_50)
data2_d_60_record_min = acc_gyr_min_order(data2_d_record_2, data2_d_label_60)
data2_d_70_record_min = acc_gyr_min_order(data2_d_record_3, data2_d_label_70)
data2_d_80_record_min = acc_gyr_min_order(data2_d_record_4, data2_d_label_80)

# data2_d_50_record_min = acc_gyr_min_order(data2_d_record_1, data2_d_label_50)
# data2_d_60_record_min = acc_gyr_min_order(data2_b_record_2, data2_d_label_60)
# data2_d_70_record_min = acc_gyr_min_order(data2_d_record_3, data2_d_label_70)
# data2_d_80_record_min = acc_gyr_min_order(data2_d_record_4, data2_d_label_80)

# a = data2_d_50_record_min
# b = data2_d_60_record_min
# c = data2_d_70_record_min
# d = data2_d_80_record_min

# e = data2_d_50_record_min
# f = data2_d_60_record_min
# g = data2_d_70_record_min
# h = data2_d_80_record_min

# min_list = pd.concat([a, b, c, d, e, f, g, h], ignore_index=True)
# min_list

## 4.5 最大ピーク周波数及び最大ピーク値

In [48]:
def fq_amp_max_order(data_record, data_label):
    #サンプル数とサンプリング周期及びカットする周波数を設定
    N = 512 # サンプル数
    dt = 0.005 # サンプリング周期(sec)
    fc = 100
    t = np.arange(0, N*dt, dt) # 時間軸
    
    f = [data_record[i]["合成加速度"].head(512) for i in range(0, len(data_label), 1)]

    # 高速フーリエ変換(FFT)
    F = np.fft.fft(f)
    F_abs = np.abs(F) # 複素数を絶対値に変換
    F_abs_amp = F_abs / N * 2 # 振幅をもとの信号に揃える(交流成分2倍)
    F_abs_amp[0] = F_abs_amp[0] / 2 # 振幅をもとの信号に揃える(直流成分非2倍)

    # 周波数軸のデータ作成
    fq = np.linspace(0, 1.0/dt, N) # 周波数軸　linspace(開始,終了,分割数)

    # フィルタリング①（周波数でカット）＊＊＊＊＊＊
    F2 = np.copy(F) # FFT結果コピー
    for i in range(0, len(data_record), 1):
        F2[i][(fq > fc)] = 0
    F2_abs = np.abs(F2) # FFTの複素数結果を絶対値に変換
    F2_abs_amp = F2_abs / N * 2 # 振幅をもとの信号に揃える(交流成分2倍)
    for i in range(0, len(data_record), 1):
        F2_abs_amp[i][0] = F2_abs_amp[i][0] / 2 # 振幅をもとの信号に揃える(直流成分非2倍)
    
    fq_amp = pd.DataFrame(fq)
    amp = [pd.DataFrame(F2_abs_amp[i])for i in range(0, len(data_label), 1)]
    a = [pd.concat([fq_amp, amp[i]], axis=1, ignore_index=True)for i in range(0, len(data_label), 1)]

    a = [pd.DataFrame(a[i]).drop(a[i].index[[0]])for i in range(0, len(data_label), 1)]
    a = [a[i][a[i][1] == np.max(a[i][1])]for i in range(0, len(data_label), 1)]
    data_record_accn_fq_amp_max = pd.concat(a, ignore_index=True)
    data_record_accn_fq_amp_max.columns = ["accn_max_fq", "accn_max_peak"]
    
    f = [data_record[i]["合成角速度"].head(512) for i in range(0, len(data_label), 1)]

    # 高速フーリエ変換(FFT)
    F = np.fft.fft(f)
    F_abs = np.abs(F) # 複素数を絶対値に変換
    F_abs_amp = F_abs / N * 2 # 振幅をもとの信号に揃える(交流成分2倍)
    F_abs_amp[0] = F_abs_amp[0] / 2 # 振幅をもとの信号に揃える(直流成分非2倍)

    # 周波数軸のデータ作成
    fq = np.linspace(0, 1.0/dt, N) # 周波数軸　linspace(開始,終了,分割数)

    # フィルタリング①（周波数でカット）＊＊＊＊＊＊
    F2 = np.copy(F) # FFT結果コピー
    for i in range(0, len(data_record), 1):
        F2[i][(fq > fc)] = 0
    F2_abs = np.abs(F2) # FFTの複素数結果を絶対値に変換
    F2_abs_amp = F2_abs / N * 2 # 振幅をもとの信号に揃える(交流成分2倍)
    for i in range(0, len(data_record), 1):
        F2_abs_amp[i][0] = F2_abs_amp[i][0] / 2 # 振幅をもとの信号に揃える(直流成分非2倍)
    
    fq_amp = pd.DataFrame(fq)
    amp = [pd.DataFrame(F2_abs_amp[i])for i in range(0, len(data_label), 1)]
    a = [pd.concat([fq_amp, amp[i]], axis=1, ignore_index=True)for i in range(0, len(data_label), 1)]

    a = [pd.DataFrame(a[i]).drop(a[i].index[[0]])for i in range(0, len(data_label), 1)]
    a = [a[i][a[i][1] == np.max(a[i][1])]for i in range(0, len(data_label), 1)]
    data_record_gyrn_fq_amp_max = pd.concat(a, ignore_index=True)
    data_record_gyrn_fq_amp_max.columns = ["gyrn_max_fq", "gyrn_max_peak"]
    
    b = pd.concat([data_record_accn_fq_amp_max, data_record_gyrn_fq_amp_max], axis=1, ignore_index=False)
    return b

data2_d_50_record_fq_amp_max = fq_amp_max_order(data2_d_record_1, data2_d_label_50)
data2_d_60_record_fq_amp_max = fq_amp_max_order(data2_d_record_2, data2_d_label_60)
data2_d_70_record_fq_amp_max = fq_amp_max_order(data2_d_record_3, data2_d_label_70)
data2_d_80_record_fq_amp_max = fq_amp_max_order(data2_d_record_4, data2_d_label_80)

# data2_d_50_record_fq_amp_max = fq_amp_max_order(data2_d_record_1, data2_d_label_50)
# data2_d_60_record_fq_amp_max = fq_amp_max_order(data2_d_record_2, data2_d_label_60)
# data2_d_70_record_fq_amp_max = fq_amp_max_order(data2_d_record_3, data2_d_label_70)
# data2_d_80_record_fq_amp_max = fq_amp_max_order(data2_d_record_4, data2_d_label_80)

# a = data2_d_50_record_fq_amp_max
# b = data2_d_60_record_fq_amp_max
# c = data2_d_70_record_fq_amp_max
# d = data2_d_80_record_fq_amp_max

# e = data2_d_50_record_fq_amp_max
# f = data2_d_60_record_fq_amp_max
# g = data2_d_70_record_fq_amp_max
# h = data2_d_80_record_fq_amp_max

# fq_amp_max_list = pd.concat([a, b, c, d, e, f, g, h], ignore_index=True)
# fq_amp_max_list

ValueError: Invalid number of FFT data points (0) specified.

In [ ]:
#全特徴量の行列の導出
data2_d_50_record_all_feature = pd.concat([data2_d_50_record_mean, data2_d_50_record_var, data2_d_50_record_max, data2_d_50_record_min, data2_d_50_record_fq_amp_max], axis=1, ignore_index=False)
data2_d_60_record_all_feature = pd.concat([data2_d_60_record_mean, data2_d_60_record_var, data2_d_60_record_max, data2_d_60_record_min, data2_d_60_record_fq_amp_max], axis=1, ignore_index=False)
data2_d_70_record_all_feature = pd.concat([data2_d_70_record_mean, data2_d_70_record_var, data2_d_70_record_max, data2_d_70_record_min, data2_d_70_record_fq_amp_max], axis=1, ignore_index=False)
data2_d_80_record_all_feature = pd.concat([data2_d_80_record_mean, data2_d_80_record_var, data2_d_80_record_max, data2_d_80_record_min, data2_d_80_record_fq_amp_max], axis=1, ignore_index=False)

# data2_d_50_record_all_feature = pd.concat([data2_d_50_record_mean, data2_d_50_record_var, data2_d_50_record_max, data2_d_50_record_min, data2_d_50_record_fq_amp_max], axis=1, ignore_index=False)
# data2_d_60_record_all_feature = pd.concat([data2_d_60_record_mean, data2_d_60_record_var, data2_d_60_record_max, data2_d_60_record_min, data2_d_60_record_fq_amp_max], axis=1, ignore_index=False)
# data2_d_70_record_all_feature = pd.concat([data2_d_70_record_mean, data2_d_70_record_var, data2_d_70_record_max, data2_d_70_record_min, data2_d_70_record_fq_amp_max], axis=1, ignore_index=False)
# data2_d_80_record_all_feature = pd.concat([data2_d_80_record_mean, data2_d_80_record_var, data2_d_80_record_max, data2_d_80_record_min, data2_d_80_record_fq_amp_max], axis=1, ignore_index=False)

In [ ]:
#編集したデータの確認(確認のためセッション1のAさんのデータを表示している)
data2_d_60_record_all_feature

In [ ]:
#各特徴量データの保存
data2_d_50_record_all_feature.to_csv('feature/D/data2_d_50_record_all_feature.csv', index=False)
data2_d_60_record_all_feature.to_csv('feature/D/data2_d_60_record_all_feature.csv', index=False)
data2_d_70_record_all_feature.to_csv('feature/D/data2_d_70_record_all_feature.csv', index=False)
data2_d_80_record_all_feature.to_csv('feature/D/data2_d_80_record_all_feature.csv', index=False)

# data2_d_50_record_all_feature.to_csv('feature/D/data2_d_50_record_all_feature.csv', index=False)
# data2_d_60_record_all_feature.to_csv('feature/D/data2_d_60_record_all_feature.csv', index=False)
# data2_d_70_record_all_feature.to_csv('feature/D/data2_d_70_record_all_feature.csv', index=False)
# data2_d_80_record_all_feature.to_csv('feature/D/data2_d_80_record_all_feature.csv', index=False)

In [ ]:
data2_d_label_50

In [ ]:
data2_d_label_50_time = data2_d_label_50.drop("emotion_level", axis=1)
data2_d_label_60_time = data2_d_label_60.drop("emotion_level", axis=1)
data2_d_label_70_time = data2_d_label_70.drop("emotion_level", axis=1)
data2_d_label_80_time = data2_d_label_80.drop("emotion_level", axis=1)

# data2_d_label_50_time = data2_d_label_50.drop("emotion_level", axis=1)
# data2_d_label_60_time = data2_d_label_60.drop("emotion_level", axis=1)
# data2_d_label_70_time = data2_d_label_70.drop("emotion_level", axis=1)
# data2_d_label_80_time = data2_d_label_80.drop("emotion_level", axis=1)

In [ ]:
data2_d_label_50_time

In [ ]:
data2_d_50_record_all_feature = pd.concat([data2_d_label_50_time, data2_d_50_record_all_feature], axis=1)
data2_d_60_record_all_feature = pd.concat([data2_d_label_60_time, data2_d_60_record_all_feature], axis=1)
data2_d_70_record_all_feature = pd.concat([data2_d_label_70_time, data2_d_70_record_all_feature], axis=1)
data2_d_80_record_all_feature = pd.concat([data2_d_label_80_time, data2_d_80_record_all_feature], axis=1)

# data2_d_50_record_all_feature = pd.concat([data2_d_label_50_time, data2_d_50_record_all_feature], axis=1)
# data2_d_60_record_all_feature = pd.concat([data2_d_label_60_time, data2_d_60_record_all_feature], axis=1)
# data2_d_70_record_all_feature = pd.concat([data2_d_label_70_time, data2_d_70_record_all_feature], axis=1)
# data2_d_80_record_all_feature = pd.concat([data2_d_label_80_time, data2_d_80_record_all_feature], axis=1)

In [ ]:
data2_d_60_record_all_feature

In [ ]:
data2_d_50_record_all_feature.to_csv('feature/D/data2_d_50_record_all_feature.csv', index=False)
data2_d_60_record_all_feature.to_csv('feature/D/data2_d_60_record_all_feature.csv', index=False)
data2_d_70_record_all_feature.to_csv('feature/D/data2_d_70_record_all_feature.csv', index=False)
data2_d_80_record_all_feature.to_csv('feature/D/data2_d_80_record_all_feature.csv', index=False)

# data2_d_50_record_all_feature.to_csv('feature/D/data2_d_50_record_all_feature.csv', index=False)
# data2_d_60_record_all_feature.to_csv('feature/D/data2_d_60_record_all_feature.csv', index=False)
# data2_d_70_record_all_feature.to_csv('feature/D/data2_d_70_record_all_feature.csv', index=False)
# data2_d_80_record_all_feature.to_csv('feature/D/data2_d_80_record_all_feature.csv', index=False)

In [ ]:
data2_d_60_record_all_feature